In [1]:
import typing

from river.tree import HoeffdingTreeRegressor, HoeffdingAdaptiveTreeRegressor, splitter
from river.datasets.synth import FriedmanDrift, Hyperplane, ConceptDriftStream
from river.datasets import Bikes, Elec2
from river.evaluate import progressive_val_score
from river.metrics import RMSE

import pandas as pd

In [2]:
class EvaluateModels:
    def __init__(self, models: typing.Dict[str, typing.Any], metric = RMSE(), instances=100000):
        self.models = models
        self.metric = metric
        self.instances = instances

    @staticmethod
    def get_dataset():
        return ConceptDriftStream(
            stream=FriedmanDrift(),
            drift_stream=FriedmanDrift(drift_type='lea')
        )

    def run_experiments(self):
        for model_name, model in self.models.items():
            print("Evaluating models - {}".format(model_name))
            progressive_val_score(
                dataset=self.get_dataset().take(self.instances),
                model=model,
                metric=RMSE(),
                print_every=20000,
                show_time=True,
                show_memory=True,
            )

    def summarize(self):
        summaries = {model_name: model.summary for model_name, model in self.models.items()}
        return pd.DataFrame(index=summaries.keys(), data=summaries.values())

In [3]:
models = {
    'qo-splitter-binary': HoeffdingTreeRegressor(splitter=splitter.QOSplitter(allow_multiway_splits=False)),
    'qo-splitter-multi': HoeffdingTreeRegressor(splitter=splitter.QOSplitter(allow_multiway_splits=True)),
    'fading-qo-splitter-.999': HoeffdingTreeRegressor(splitter=splitter.FadingQOSplitter(allow_multiway_splits=True)),
    'fading-qo-splitter-clone': HoeffdingTreeRegressor(splitter=splitter.FadingQOSplitter(allow_multiway_splits=True)),
    'adaptive': HoeffdingAdaptiveTreeRegressor()
}

In [4]:
evaluation = EvaluateModels(
    models=models,
)
evaluation.run_experiments()

Evaluating models - qo-splitter-binary
[20,000] RMSE: 2.474949 – 00:00:01 – 1.58 MiB
[40,000] RMSE: 2.381478 – 00:00:03 – 3.26 MiB
[60,000] RMSE: 2.341639 – 00:00:05 – 4.67 MiB
[80,000] RMSE: 2.331885 – 00:00:07 – 5.79 MiB
[100,000] RMSE: 2.315637 – 00:00:09 – 7.08 MiB
Evaluating models - qo-splitter-multi
[20,000] RMSE: 2.487789 – 00:00:01 – 2.07 MiB
[40,000] RMSE: 2.318995 – 00:00:03 – 3.77 MiB
[60,000] RMSE: 2.271053 – 00:00:05 – 5.12 MiB
[80,000] RMSE: 2.26959 – 00:00:07 – 9.33 MiB
[100,000] RMSE: 2.236478 – 00:00:09 – 14.86 MiB
Evaluating models - fading-qo-splitter-.999
[20,000] RMSE: 2.429079 – 00:00:01 – 1.45 MiB
[40,000] RMSE: 2.300205 – 00:00:03 – 3.1 MiB
[60,000] RMSE: 2.248728 – 00:00:05 – 5.02 MiB
[80,000] RMSE: 2.232955 – 00:00:08 – 6.8 MiB
[100,000] RMSE: 2.204936 – 00:00:10 – 9.07 MiB
Evaluating models - fading-qo-splitter-clone
[20,000] RMSE: 2.463899 – 00:00:03 – 1.61 MiB
[40,000] RMSE: 2.321049 – 00:00:05 – 3.52 MiB
[60,000] RMSE: 2.245163 – 00:00:07 – 4.98 MiB
[80,0

In [5]:
valuation.summarize()

,n_nodes,n_branches,n_leaves,n_active_leaves,n_inactive_leaves,height,total_observed_weight,n_alternate_trees,n_pruned_alternate_trees,n_switch_alternate_trees
qo-splitter-binary,485,242,243,243,0,16,100000.0,NaN,NaN,NaN
qo-splitter-multi,757,189,568,568,0,6,100000.0,NaN,NaN,NaN
fading-qo-splitter-.999,509,218,291,291,0,13,100000.0,NaN,NaN,NaN
fading-qo-splitter-clone,508,220,288,288,0,11,100000.0,NaN,NaN,NaN
adaptive,61,30,31,420,0,10,100000.0,31.0,6.0,0.0
